# 더 나은 모델을 만들기 위한 기법들

학습률 계획  
가중치 초기화  
과적합 방지  
드롭아웃  
조기종료  
...


# Fashion MNIST 신경망 예제에 기법 적용하기

In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import regularizers

import os
import numpy as np
import tensorflow as tf
import keras

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

1. **데이터셋을 불러옵니다.**

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
print(X_train.shape, X_test.shape)

4422102/4422102 [==============================] - 0s 0us/step
(60000, 28, 28) (10000, 28, 28)


2. **데이터를 정규화(Normalization)합니다.**

In [4]:
X_train = X_train / 255.
X_test = X_test / 255.

3. **레이블의 개수와 형태를 확인합니다.**

In [5]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

4. **신경망 모델을 구축하고 Compile 합니다.**

구축 과정에서 위에서 학습하였던 **Weight Decay(가중치 감소), Dropout(드롭아웃)**을 적용해봅시다.

In [6]:
# 기본적인 신경망을 만드는 코드
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64,
          kernel_regularizer=regularizers.l2(0.01),
          activity_regularizer=regularizers.l1(0.01)),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

Compile 설정에서 위에서 학습하였던 **Learning rate Decay(학습률 감소)**를 적용해봅니다.

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1 = 0.89)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 64)                50240     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


5. **신경망 모델을 학습합니다.**

먼저 학습 과정에서 **Early Stopping(조기 종료)**를 적용할 수 있도록<br/>
파라미터 저장 경로와 조기 종료 옵션을 설정하여 줍니다.

In [9]:
# 파라미터 저장 경로를 설정하는 코드입니다.
checkpoint_filepath = "FMbest.hdf5"

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

In [10]:
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

In [11]:
model.fit(X_train, y_train, batch_size=32, epochs=30, verbose=1, 
          validation_data=(X_test,y_test), 
          callbacks=[early_stop, save_best])

Epoch 1/30
1860/1875 [============================>.] - ETA: 0s - loss: 1.2092 - accuracy: 0.7720
Epoch 1: val_loss improved from inf to 0.86188, saving model to FMbest.hdf5
1875/1875 [==============================] - 12s 3ms/step - loss: 1.2071 - accuracy: 0.7722 - val_loss: 0.8619 - val_accuracy: 0.8074
Epoch 2/30
1869/1875 [============================>.] - ETA: 0s - loss: 0.9237 - accuracy: 0.7899
Epoch 2: val_loss did not improve from 0.86188
1875/1875 [==============================] - 6s 3ms/step - loss: 0.9237 - accuracy: 0.7898 - val_loss: 0.8774 - val_accuracy: 0.7977
Epoch 3/30
1859/1875 [============================>.] - ETA: 0s - loss: 0.9053 - accuracy: 0.7915
Epoch 3: val_loss did not improve from 0.86188
1875/1875 [==============================] - 6s 3ms/step - loss: 0.9066 - accuracy: 0.7911 - val_loss: 0.8657 - val_accuracy: 0.7979
Epoch 4/30
1873/1875 [============================>.] - ETA: 0s - loss: 0.8977 - accuracy: 0.7927
Epoch 4: val_loss improved from 0.8618

6. **조기종료 직전의 모델을 사용하여 평가를 진행합니다.**

In [12]:
model.predict(X_test[0:1])

1/1 [==============================] - 0s 87ms/step


array([[8.0170983e-05, 1.1109122e-04, 2.0825553e-04, 6.3865515e-04,
        1.4710909e-04, 2.7330446e-01, 2.3345777e-04, 2.5552991e-01,
        4.9631735e-03, 4.6478376e-01]], dtype=float32)

In [13]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.8177 - accuracy: 0.8122 - 615ms/epoch - 2ms/step


7. **콜백(Callback)에 의해 Best 모델의 파라미터가 제대로 저장되었는지 확인하고 해당 모델로 평가를 진행합니다.**

In [14]:
!ls 

FMbest.hdf5  sample_data


저장된 경로로부터 모델의 파라미터(가중치)를 불러옵니다.

In [15]:
model.load_weights(checkpoint_filepath)

불러온 모델을 사용하여 평가를 수행합니다.

In [16]:
model.predict(X_test[0:1])

1/1 [==============================] - 0s 22ms/step


array([[3.2536616e-04, 1.9320493e-04, 7.4596016e-04, 1.8528182e-03,
        5.8786123e-04, 6.4009041e-02, 5.9805281e-04, 3.1626484e-01,
        2.0162420e-02, 5.9526044e-01]], dtype=float32)

In [17]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 0.7835 - accuracy: 0.8197


# Neural Network Framework (Keras)

### 케라스 라이브러리를 사용하여 Multi-Layer Perceptron 모델을 CIFAR100 데이터에 적용해보세요.

- 시드를 고정하십시오.
- 데이터를 Noramlized 해줍니다. 
- 케라스에서 모델은 다음과 같이 고정합니다. 
- 은닉층의 활성함수는 ReLU를 사용합니다.
- 단계별로 오늘 배운 규제방법을 적용해봅니다. 

In [18]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import random
import os

In [19]:
import tensorflow
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3334654592427693766
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14378926080
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6376249490395125852
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


### 1) Base model을 제작합니다.

In [20]:
# seed를 고정합니다.
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)
# os.environ["PYTHONHASHSEED"] = str(1)
# os.environ['TF_DETERMINISTIC_OPS'] = str(1)
tf.random.set_seed(1)

# 데이터 불러오기
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

print(X_train.shape)

# 정규화(전처리)

X_train = X_train/ 255.0
X_test = X_test/ 255.0

print(X_train.shape)
# 변수 설정을 따로 하는 방법을 적용하기 위한 코드입니다. 
batch_size = 100
epochs_max = 20

# model
tf.random.set_seed(1)
model = Sequential()
model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(100, activation='softmax'))

tf.random.set_seed(1)
# 컴파일 단계, 옵티마이저와 손실함수, 측정지표를 연결해서 계산 그래프를 구성함
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

results = model.fit(X_train, y_train, epochs=epochs_max, batch_size=batch_size, verbose=1, validation_data=(X_test,y_test))

169001437/169001437 [==============================] - 3s 0us/step
(50000, 32, 32, 3)
(50000, 32, 32, 3)
Epoch 1/20
500/500 [==============================] - 3s 4ms/step - loss: 4.1182 - accuracy: 0.0727 - val_loss: 3.8397 - val_accuracy: 0.1121
Epoch 2/20
500/500 [==============================] - 2s 3ms/step - loss: 3.7378 - accuracy: 0.1279 - val_loss: 3.6938 - val_accuracy: 0.1415
Epoch 3/20
500/500 [==============================] - 2s 3ms/step - loss: 3.5827 - accuracy: 0.1570 - val_loss: 3.6031 - val_accuracy: 0.1613
Epoch 4/20
500/500 [==============================] - 2s 3ms/step - loss: 3.4825 - accuracy: 0.1741 - val_loss: 3.5155 - val_accuracy: 0.1764
Epoch 5/20
500/500 [==============================] - 2s 4ms/step - loss: 3.4188 - accuracy: 0.1862 - val_loss: 3.4556 - val_accuracy: 0.1895
Epoch 6/20
500/500 [==============================] - 2s 4ms/step - loss: 3.3498 - accuracy: 0.1985 - val_loss: 3.4228 - val_accuracy: 0.1930
Epoch 7/20
500/500 [=======================

In [21]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 3.2907 - accuracy: 0.2226


### 2) + Weight Decay

In [22]:
# seed를 고정합니다.
random.seed(1)
np.random.seed(1)

tf.random.set_seed(1)

# 데이터 불러오기
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

# 정규화(전처리)
X_train = X_train/ 255.0
X_test = X_test/ 255.0

# 변수 설정을 따로 하는 방법을 적용하기 위한 코드입니다. 
batch_size = 100
epochs_max = 20

# model
weight_decay = 1e-5

model = Sequential()
model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(128, activation='relu', 
                kernel_regularizer=regularizers.l2(weight_decay),   
                activity_regularizer=regularizers.l1(weight_decay)))
model.add(Dense(128, activation='relu'))
model.add(Dense(100, activation='softmax'))

# 컴파일 단계, 옵티마이저와 손실함수, 측정지표를 연결해서 계산 그래프를 구성함
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

results = model.fit(X_train, y_train, epochs=epochs_max, batch_size=batch_size, verbose=1, validation_data=(X_test,y_test))

Epoch 1/20
500/500 [==============================] - 3s 4ms/step - loss: 4.1290 - accuracy: 0.0713 - val_loss: 3.8451 - val_accuracy: 0.1114
Epoch 2/20
500/500 [==============================] - 2s 5ms/step - loss: 3.7463 - accuracy: 0.1283 - val_loss: 3.7237 - val_accuracy: 0.1336
Epoch 3/20
500/500 [==============================] - 2s 4ms/step - loss: 3.6132 - accuracy: 0.1511 - val_loss: 3.6940 - val_accuracy: 0.1404
Epoch 4/20
500/500 [==============================] - 2s 4ms/step - loss: 3.5303 - accuracy: 0.1645 - val_loss: 3.5305 - val_accuracy: 0.1770
Epoch 5/20
500/500 [==============================] - 2s 4ms/step - loss: 3.4585 - accuracy: 0.1780 - val_loss: 3.5226 - val_accuracy: 0.1767
Epoch 6/20
500/500 [==============================] - 2s 4ms/step - loss: 3.3974 - accuracy: 0.1904 - val_loss: 3.4607 - val_accuracy: 0.1896
Epoch 7/20
500/500 [==============================] - 2s 4ms/step - loss: 3.3514 - accuracy: 0.1977 - val_loss: 3.4967 - val_accuracy: 0.1813
Epoch 

In [23]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

313/313 [==============================] - 1s 3ms/step - loss: 3.3495 - accuracy: 0.2101


### 3) + Dropout

In [24]:
# seed를 고정합니다.
random.seed(1)
np.random.seed(1)

tf.random.set_seed(1)

# 데이터 불러오기
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

# 정규화(전처리)
X_train = X_train/ 255.0
X_test = X_test/ 255.0

# 변수 설정을 따로 하는 방법을 적용하기 위한 코드입니다. 
batch_size = 100
epochs_max = 20

# model
model = Sequential()
model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(128*1.1, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dense(100, activation='softmax'))

# 컴파일 단계, 옵티마이저와 손실함수, 측정지표를 연결해서 계산 그래프를 구성함
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

results = model.fit(X_train, y_train, epochs=epochs_max, batch_size=batch_size, verbose=1, validation_data=(X_test,y_test))


Epoch 1/20
500/500 [==============================] - 3s 4ms/step - loss: 4.2795 - accuracy: 0.0442 - val_loss: 4.0390 - val_accuracy: 0.0699
Epoch 2/20
500/500 [==============================] - 2s 3ms/step - loss: 4.0321 - accuracy: 0.0728 - val_loss: 3.9263 - val_accuracy: 0.0965
Epoch 3/20
500/500 [==============================] - 2s 4ms/step - loss: 3.9320 - accuracy: 0.0893 - val_loss: 3.8381 - val_accuracy: 0.1112
Epoch 4/20
500/500 [==============================] - 2s 4ms/step - loss: 3.8640 - accuracy: 0.1023 - val_loss: 3.7654 - val_accuracy: 0.1253
Epoch 5/20
500/500 [==============================] - 2s 3ms/step - loss: 3.7928 - accuracy: 0.1148 - val_loss: 3.6909 - val_accuracy: 0.1369
Epoch 6/20
500/500 [==============================] - 2s 4ms/step - loss: 3.7512 - accuracy: 0.1217 - val_loss: 3.6631 - val_accuracy: 0.1412
Epoch 7/20
500/500 [==============================] - 2s 3ms/step - loss: 3.7191 - accuracy: 0.1247 - val_loss: 3.6535 - val_accuracy: 0.1448
Epoch 

### 4) + Early Stopping

1. Early Stopping을 적용해봅시다. 멈추는 기준을 `val_loss(검증 데이터셋의 loss 값)`로 하고 loss가 Best 값보다 5번 높아질 때 Stop 하도록 설정합니다.

2. Best 모델을 저장해봅시다. Best 모델 역시 멈추는 기준을 `val_loss(검증 데이터셋의 loss 값)`로 하고 `save_best_only=True, save_weights_only=True` 로 설정합니다.

In [25]:
# seed를 고정합니다.
random.seed(1)
np.random.seed(1)

tf.random.set_seed(1)


# 데이터 불러오기
(X_train, y_train), (X_test, y_test) = cifar100.load_data()


# 정규화(전처리)
X_train = X_train/ 255.0
X_test = X_test/ 255.0


# 학습시킨 데이터를 저장시키기 위한 코드입니다. 
checkpoint_filepath = "FMbest.hdf5"


# 변수 설정을 따로 하는 방법을 적용하기 위한 코드입니다. 
batch_size = 100
epochs_max = 50


model = Sequential()
model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(100, activation='softmax'))

# 컴파일 단계, 옵티마이저와 손실함수, 측정지표를 연결해서 계산 그래프를 구성함
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                  
# early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1)

# Validation Set을 기준으로 가장 최적의 모델을 찾기
save_best = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

results = model.fit(X_train, y_train, batch_size=32, epochs=epochs_max, verbose=1, 
          validation_data=(X_test,y_test), 
          callbacks=[early_stop, save_best])

Epoch 1/50
1559/1563 [============================>.] - ETA: 0s - loss: 4.1069 - accuracy: 0.0671
Epoch 1: val_loss improved from inf to 3.93226, saving model to FMbest.hdf5
1563/1563 [==============================] - 7s 4ms/step - loss: 4.1058 - accuracy: 0.0672 - val_loss: 3.9323 - val_accuracy: 0.0911
Epoch 2/50
1550/1563 [============================>.] - ETA: 0s - loss: 3.8411 - accuracy: 0.1062
Epoch 2: val_loss improved from 3.93226 to 3.82695, saving model to FMbest.hdf5
1563/1563 [==============================] - 6s 4ms/step - loss: 3.8407 - accuracy: 0.1063 - val_loss: 3.8269 - val_accuracy: 0.1115
Epoch 3/50
1553/1563 [============================>.] - ETA: 0s - loss: 3.7509 - accuracy: 0.1210
Epoch 3: val_loss improved from 3.82695 to 3.72216, saving model to FMbest.hdf5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.7511 - accuracy: 0.1210 - val_loss: 3.7222 - val_accuracy: 0.1342
Epoch 4/50
1553/1563 [============================>.] - ETA: 0s - loss:

In [26]:
# 학습된 모델을 이용하여 테스트하는 코드
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 3.5476 - accuracy: 0.1617


In [27]:
model.load_weights(checkpoint_filepath)

# best model을 이용한 테스트 데이터 예측 정확도 재확인 코드
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 3.5473 - accuracy: 0.1622
